In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import key

In [2]:
# Store the CSV into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Upernavik,GL,2021-01-19 02:13:58,72.7868,-56.1549,7.07,78,10,7.83
1,1,Niort,FR,2021-01-19 02:09:18,46.3333,-0.3333,34.00,88,96,8.50
2,2,Luderitz,NaN,2021-01-19 02:13:58,-26.6481,15.1594,64.92,84,49,11.97
3,3,Rikitea,PF,2021-01-19 02:13:58,-23.1203,-134.9692,78.48,74,72,14.41
4,4,Saint-Philippe,RE,2021-01-19 02:13:58,-21.3585,55.7679,78.80,69,0,4.61


In [3]:
# Get data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use Google API key
gmaps.configure(api_key=key)

In [8]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Or add [max(temp, 0) for temp in max_temp] right in the code below

In [17]:
# Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
# max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, max_intensity=300, point_radius=4, dissipating=False)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))